In [246]:
import warnings
warnings.filterwarnings("ignore")
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Campero/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [170]:
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la
#from numpy import ndarray as nda
import time
import pandas as pd
import json
import contractions

In [3]:
docs_path = 'data/tweet_document_ids_map.csv'
with open(docs_path) as fp:
    lines = fp.read().split("\n")
lines = [l for l in lines if l != ""]
print("Total number of tweet documents in the corpus: {}".format(len(lines)))

tweet2doc = dict()
doc2tweet = dict()

for line in lines:
    docID, tweetID = tuple(line.split("\t"))
    tweet2doc[tweetID] = docID
    doc2tweet[docID] = tweetID

Total number of tweet documents in the corpus: 4000


In [4]:
tweets_path = 'data/tw_hurricane_data.json'
with open(tweets_path) as fp:
    tweets = fp.read().split("\n")
tweets = [t for t in tweets if t != ""]
print("Total number of tweets in the corpus: {}".format(len(tweets)))

Total number of tweets in the corpus: 4000


In [5]:
tweets_json=[]
for t in tweets:
    tweets_json.append(json.loads(t))

In [6]:
lengths = set()
for c in tweets_json:
    entities = c["entities"]
    c["hashtags"] = []
    for hashtag in entities["hashtags"]:
        c["hashtags"].append(hashtag["text"])
    c["url"] = None
    if "media" in entities.keys():
        if "url" in entities["media"][0].keys():
            c["url"] = entities["media"][0]["url"]
    #c["urls"] = entities["urls"]
    c["username"] = c["user"]["name"]
    
columns = {
    "full_text":"Tweet",
    "username":"Username",
    "created_at":"Date",
    "hashtags":"Hashtags",
    "favorite_count":"Likes",
    "retweet_count":"Retweets",
    "url":"Url"
}

dt_tweets = pd.DataFrame.from_dict(data=tweets_json)[columns.keys()].rename(columns=columns)
dt_tweets.head()

,Tweet,Username,Date,Hashtags,Likes,Retweets,Url
0,So this will keep spinning over us until 7 pm…...,Suz👻,Fri Sep 30 18:39:08 +0000 2022,[HurricaneIan],0,0,https://t.co/VROTxNS9rz
1,Our hearts go out to all those affected by #Hu...,Lytx,Fri Sep 30 18:39:01 +0000 2022,[HurricaneIan],0,0,None
2,Kissimmee neighborhood off of Michigan Ave. \n...,Christopher Heath,Fri Sep 30 18:38:58 +0000 2022,[HurricaneIan],0,0,https://t.co/jf7zseg0Fe
3,I have this one tree in my backyard that scare...,alex ✨,Fri Sep 30 18:38:57 +0000 2022,"[scwx, HurricaneIan]",0,0,None
4,@AshleyRuizWx @Stephan89441722 @lilmizzheidi @...,Tess 💋,Fri Sep 30 18:38:53 +0000 2022,[HurricaneIan],0,0,None


In [243]:
def build_terms(line):
    """
    Preprocess the tweet content removing stop words, stemming,
    transforming in lowercase, removing special characters [#, @, .] and removing tweet url 
    (since it is included in another column on the dataframe)
    
    return tokenized tweet (list of words after applying the previous steps).
    
    Argument:
    line -- string (tweet) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """
    ## START CODE
    line = line.lower() ## Transform in lowercase
    line = re.sub(r"[^A-Za-z 0-9 ']+", '', line)
    stop_words = set(stopwords.words("english"))
    line = ' '.join([contractions.fix(x) for x in line.split(' ')])
    line = re.sub("'", '', line)
    line = line.split(r'\s+')
    line = [x for x in line if x and x not in stop_words]
    line = filter(lambda x:x[0:5]!='https', line)
    line = [x for x in line]
    ps = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    line = [lemmatizer.lemmatize(x) for x in line]
    line = [ps.stem(x) for x in line]
    
    ## END CODE
    return ' '.join(line)

In [244]:
build_terms("Where are we going alex ✨? I'm  going to buy the gardener's cats' food from this link https://hola.com")

['go', 'alex', 'go', 'buy', 'garden', 'cat', 'food', 'link']

In [245]:
build_terms(dt_tweets.iloc[3]['Tweet'])

NameError: name 'dt_tweets' is not defined

In [194]:
tweets_processed = dt_tweets.copy()
for index, row in tweets_processed.iterrows():
    tweets_processed["Tweet"][index] = build_terms(row['Tweet'])
tweets_processed.head()

NameError: name 'dt_tweets' is not defined

In [15]:
type(tweet2doc.keys())

dict_keys

In [16]:
tweets_processed["doc_id"]=[tweet2doc[str(x["id"])] for x in tweets_json]

In [17]:
tweets_processed.head()

,Tweet,Username,Date,Hashtags,Likes,Retweets,Url,doc_id
0,"[keep, spin, us, 7, pm…go, away, alreadi, hurr...",Suz👻,Fri Sep 30 18:39:08 +0000 2022,[HurricaneIan],0,0,https://t.co/VROTxNS9rz,doc_1
1,"[heart, go, affect, hurricaneian, wish, everyo...",Lytx,Fri Sep 30 18:39:01 +0000 2022,[HurricaneIan],0,0,None,doc_2
2,"[kissimme, neighborhood, michigan, ave, hurric...",Christopher Heath,Fri Sep 30 18:38:58 +0000 2022,[HurricaneIan],0,0,https://t.co/jf7zseg0Fe,doc_3
3,"[one, tree, backyard, scare, poltergeist, tree...",alex ✨,Fri Sep 30 18:38:57 +0000 2022,"[scwx, HurricaneIan]",0,0,None,doc_4
4,"[ashleyruizwx, stephan89441722, lilmizzheidi, ...",Tess 💋,Fri Sep 30 18:38:53 +0000 2022,[HurricaneIan],0,0,None,doc_5
